In [2]:
import requests
from multiprocessing.dummy import Pool as ThreadPool
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
def moneyfrom(in_str):
    get_data = [int(s) for s in 
                in_str.replace("-", " ").replace("–", " ").split(" ") if s.isdigit()]
    if len(get_data) == 2:
        if get_data[0] == 700:
            return 2
        if get_data[0] == 1500:
            return 3
    else:
        if get_data[0] == 700:
            return 1
        if get_data[0] == 2500:
            return 4

def getInfoFromResponse(data, resp, num_resp):
    if resp.status_code != 200:
        print("Error", num_resp)
        return
    if num_resp % 20 == 0:   
        print(resp, num_resp)
    bs = BeautifulSoup(resp.text, 'lxml')
    for item in bs.find_all('div', 'places_info'): 
        find_name = item.find('span', 'places_name')
        if find_name is not None: 
            name = find_name.text.strip()
        else: 
            name = None

        find_address = item.find('span', 'places_address')
        if find_address is not None:     
            address = find_address.text.strip()
        else: 
            address = None

        find_metro = item.find('span', 'places_metro')
        if find_metro is not None:     
            metro = find_metro.text.strip()[3:]
        else: 
            metro = None

        find_rating = item.find('div', 'rating_number')
        if find_rating is not None: 
            rating = item.find('div', 'rating_number').text.strip()
        else:
            rating = None
        
        find_money = item.find('span', "range s-tooltip")
        if find_money is not None:
            money = moneyfrom(find_money.get("data-title"))
        else:
            money = None

        data.append((name, address, metro, rating, money))

In [4]:
data = list()
FIRST_URL = "https://www.afisha.ru/msk/restaurants/restaurant_list/"
first_resp = requests.get(FIRST_URL)
getInfoFromResponse(data, first_resp, 1)

t = ThreadPool(2)
start_page = 2
end_page = 453
r = t.map(lambda x: getInfoFromResponse(data, 
                    requests.get("https://www.afisha.ru/msk/restaurants/restaurant_list/page%d/?view=list" % x), x), 
          range(start_page, end_page))
t.close()
t.join()

<Response [200]> 60
<Response [200]> 20
<Response [200]> 80
<Response [200]> 40
<Response [200]> 100
<Response [200]> 120
<Response [200]> 180
<Response [200]> 140
<Response [200]> 200
<Response [200]> 160
<Response [200]> 220
<Response [200]> 240
<Response [200]> 300
<Response [200]> 260
<Response [200]> 320
<Response [200]> 280
<Response [200]> 340
<Response [200]> 360
<Response [200]> 420
Error 428
Error 429
Error 430
Error 431
Error 432
Error 433
Error 434
Error 435
Error 436
Error 437
Error 438
Error 439
Error 440
Error 441
Error 442
Error 443
Error 444
Error 445
Error 446
Error 447
Error 448
Error 449
Error 450
Error 451
Error 452
<Response [200]> 380
<Response [200]> 400


In [5]:
df = pd.DataFrame(np.array(data, dtype = object), columns=["Name", "Address", "Metro", "Rating", "Money"])

In [6]:
df.to_csv("parsed_afisha.csv", sep=',', encoding='utf-8')

In [11]:
df = df.from_csv("parsed_afisha.csv", sep=',', encoding='utf-8')

In [14]:
df = df.sort_values('Rating', ascending=False).dropna()

In [15]:
df.shape

(1246, 5)

In [18]:
df.head(10)

,Name,Address,Metro,Rating,Money
1,Beer Happens,"Сретенка, 24/2, стр. 1",Сухаревская,9.1,2.0
2,Винный базар на Комсомольском,"Комсомольский просп., 14/1, корп. 2",Парк культуры,9.1,2.0
3,Grammy's,"Кутузовский просп., 2/1, стр. 6, в Конгресс-па...",Киевская,9.0,3.0
4,Mitzva Bar,"Пятницкая, 3/4, стр. 1","Новокузнецкая, Третьяковская",9.0,2.0
5,Боэми,"Рочдельская, 11/5","Краснопресненская, Баррикадная",9.0,3.0
6,Lumberjack Bar,"Б.Спасоглинищевский пер., 3, стр. 5",Китай-город,9.0,3.0
7,Бутчер,"Лубянский пр., 15/2",Китай-город,9.0,4.0
8,Лейка,"Головинское ш., 5, ТЦ «Водный», 3 этаж",Водный стадион,9.0,2.0
12,Let's Rock Bar,"Кузнецкий Мост, 4","Охотный Ряд, Театральная, Площадь Революции",8.9,2.0
13,AQ Chicken,"Трехпрудный пер., 11/13, стр. 2","Тверская, Пушкинская, Чеховская",8.9,3.0
